# 0.1. IMPORTS

In [31]:
import pandas as pd
import numpy as np
from statsmodels.stats.gof import chisquare_effectsize
from statsmodels.stats.power import TTestIndPower
from scipy import stats
from itertools import combinations
from statsmodels.sandbox.stats.multicomp import multipletests

# 1.0. LOADING DATA

In [2]:
d= {'variant':['interact','connect', 'learn', 'help', 'services'],
    'visits': [10283, 2742, 2747, 3180, 2064],
    'click_all': [3714, 1587, 1652, 1717, 1348],
    'click_link': [42, 53, 21, 38, 45]}

data = pd.DataFrame(d) # está criando um dataframe

data['conversion'] = data["click_link"] / data["click_all"]
data

,variant,visits,click_all,click_link,conversion
0,interact,10283,3714,42,0.011309
1,connect,2742,1587,53,0.033396
2,learn,2747,1652,21,0.012712
3,help,3180,1717,38,0.022132
4,services,2064,1348,45,0.033383


# 2.0. Design de Experimentos

## 2.1. Formulação das hipóteses

In [3]:
# H0: Não há nenhuma diferença ente o CTR(Click-Through Rate) das variantes das páginas
# H1: Há uma diferença entre o CTR das variantes das páginas
#
# Test Bi-Caudal

## 2.2. Parâmetros do Experimento

In [ ]:
k = len( data['click_link']) # tamanho do dataset da coluna click_link
actual_dist = data['click_link'] / data['click_link'].sum() # proporção de visualização de cada página
expected_dist = [1 /k] * k # significa que k=5, ou seja, 1/k = 1/5 =0.2, então, cria uma lista que aparece 5vezes o 0.2

In [8]:
k

5

In [7]:
actual_dist

0    0.211055
1    0.266332
2    0.105528
3    0.190955
4    0.226131
Name: click_link, dtype: float64

In [9]:
expected_dist

[0.2, 0.2, 0.2, 0.2, 0.2]

In [ ]:
effect_size = chisquare_effectsize( expected_dist, actual_dist) # tamanho do efeito
alpha = 0.05 # nivel de significância para rejeitar a hipótese nula
power = 0.80 # para aceitar a hipótese nula

sample_size = TTestIndPower().solve_power(
                effect_size = effect_size,
                alpha = alpha,
                power = power,
            )

sample_size = np.ceil(sample_size).astype(int) # o ceil é para fazer o arredondamento
print(f'Minimum Sample Size per Variant: {sample_size}')
print(f'Total Sample Size: {k*sample_size}')

Minimum Sample Size per Variant: 222
Total Sample Size: 1110


# 3.0. Aplicação do Teste Chi Squared

In [ ]:
data['no_click_link'] = data['click_all'] - data['click_link'] # os links que não teve click

df = data[['variant', 'click_link', 'no_click_link']]
df = df.set_index('variant') # ta renomeando as linhas com a variável variant
df


,click_link,no_click_link
variant,,
interact,42,3672
connect,53,1534
learn,21,1631
help,38,1679
services,45,1303


In [23]:
chi2, p_value, dof, ex = stats.chi2_contingency(df)
print(f'Chi Squared: {chi2} - p-value: {p_value}')

Chi Squared: 46.33660181942126 - p-value: 2.0959498129984567e-09


# 4.0. Post-hoc Testing

In [33]:
all_combinations = list(combinations(df.index, 2)) # combinação de 2 em 2 da coluna variant

p_values = []

for comb in all_combinations:
    new_df = df[(df.index == comb[0]) | (df.index == comb[1])] # df.index foi criada na parte de cima df = df.set_index('variant')
    chi2, p_value, dof, ex = stats.chi2_contingency(new_df)
    p_values.append(p_value)
    
reject_list, correct_p_values = multipletests(p_values, method='bonferroni')[:2]
    

In [34]:
for comb, p_val, corr_p_val, reject in zip(all_combinations, p_values, correct_p_values, reject_list):
    print(f'\n{comb}: p_value: {p_val}; corrected_p_value: {corr_p_val}; reject: {reject}')


('interact', 'connect'): p_value: 5.3676772349808135e-08; corrected_p_value: 5.367677234980813e-07; reject: True

('interact', 'learn'): p_value: 0.7616980743361713; corrected_p_value: 1.0; reject: False

('interact', 'help'): p_value: 0.0031030587017400212; corrected_p_value: 0.03103058701740021; reject: True

('interact', 'services'): p_value: 1.798089447385411e-07; corrected_p_value: 1.7980894473854111e-06; reject: True

('connect', 'learn'): p_value: 0.00013292868361715983; corrected_p_value: 0.0013292868361715984; reject: True

('connect', 'help'): p_value: 0.06144184057612575; corrected_p_value: 0.6144184057612575; reject: False

('connect', 'services'): p_value: 1.0; corrected_p_value: 1.0; reject: False

('learn', 'help'): p_value: 0.0508958228881819; corrected_p_value: 0.5089582288818191; reject: False

('learn', 'services'): p_value: 0.00020374035733741825; corrected_p_value: 0.0020374035733741825; reject: True

('help', 'services'): p_value: 0.07301998638337415; corrected_p

In [35]:
# Interact x Connect -> Há uma diferença, dependência -> p_value = 5.367677234980813e-07
# Interact x Services -> Há uma diferença, dependência -> p_value = 1.7980894473854111e-06
# Interact x Help -> Há uma diferença, dependência -> p_value = 0.03103058701740021

# Connect x Services -> Não há uma diferença
# Connect x Help -> Não há uma diferença
# Help x Services -> Não há uma diferença